In [1]:
## Import Libraries
# General system libraries
import os
import numpy as np
import pandas 
from time import time
from IPython.display import Image

# Multiprocessing
import multiprocessing

# DNA/RNA Analysis Libraries (Biopython, ViennaRNA, pysster)
# Biopython Lib
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import Alphabet, DNAAlphabet, ProteinAlphabet
from Bio.Alphabet import generic_rna, generic_dna, generic_protein, IUPAC
# ViennaRNA Lib
import RNA
# pysster Lib
from pysster import utils
from pysster.Data import Data
from pysster.Grid_Search import Grid_Search
from pysster.One_Hot_Encoder import One_Hot_Encoder
from pysster.Alphabet_Encoder import Alphabet_Encoder

# Import TPOT libs
#from tpot import TPOTRegressor

# Import sklearn libs
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, mean_absolute_error
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.metrics import median_absolute_error, r2_score

# Math & Visualization Libs
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

# Progress Bar
from tqdm import tqdm

# Warnings
import warnings

import subprocess


Using TensorFlow backend.


In [2]:
df = pandas.DataFrame(pandas.read_csv("toeholddatasetprocPEARFinalQC_2019Mar1_SalisRBS.csv"))
df.loc[:,"loop1"] = "AACCAAACACACAAACGCAC"
x = df.loc[:,"trigger":"post_linker"]
y = pandas.DataFrame("G" + x.loc[:,"trigger"]+x.loc[:,"loop1"]+x.loc[:,"switch1"]+x.loc[:,"loop2"]+x.loc[:,"stem1"]+x.loc[:,"atg"]+x.loc[:,"stem2"]+x.loc[:,"linker"]+x.loc[:,"post_linker"])
y.columns = ["full"]
df = pandas.concat([df,y],axis=1)


FileNotFoundError: File b'toeholddatasetprocPEARFinalQC_2019Mar1_SalisRBS.csv' does not exist

In [ ]:
subdf = pandas.DataFrame()
subdf_full = pandas.DataFrame()
subdf['sub_switchoff'] = "GGG" + df.loc[:,"full"].str[31:146]
subdf['sub_switchon'] = "GG" + df.loc[:,"full"].str[0:146]
#subdf_full['sub_switchoff'] = "GGG" + df.loc[:,"full"].str[31:146]
#subdf_full['sub_switchon'] = "GG" + df.loc[:,"full"].str[0:146]
#subdf['sub_toeholdoff'] = "GG" + df.loc[:,"full"].str[0:31]
subdf['sub_toeholdon'] = "GG" + df.loc[:,"full"].str[0:63]
subdf['sub_stem'] = df.loc[:,"full"].str[63:110]
#subdf['sub_ascendingstem'] = df.loc[:,"full"].str[63:92]
#subdf['sub_descendingstem'] = df.loc[:,"full"].str[81:110]
subdf['sub_stemtop'] = df.loc[:,"full"].str[75:98]
#subdf['sub_rbsgfp'] = df.loc[:,"full"].str[81:146]
#subdf['sub_rbslinker'] = df.loc[:,"full"].str[81:136]


In [3]:
#def mfe(x):
#    (ss,mfe) = RNA.fold_compound(x).mfe()
#    return mfe
def kinfold(x):
    s = "\"" + x + "\""
    #p = "| /usr/local/bin/Kinfold --time 1000 | grep X1"
    p = "| /usr/local/bin/Kinfold --time 10000000 | grep X1 | perl -ne 'chomp;@a=split(/\s+/);print $a[2] . \"\\n\";' | perl -e '@a=<>;$s=0;for(my $i=0;$i<scalar(@a);$i++){$s+=$a[$i];}print $s/scalar(@a);'"
    #p = "| /usr/local/bin/Kinfold --time 1000 | grep O | perl -ne 'chomp;@a=split(/\s+/);print $a[2] . \"\\n\";' | perl -e '@a=<>;$s=0;for(my $i=0;$i<scalar(@a);$i++){$s+=$a[$i];}print $s/scalar(@a);'"
    cmd = "echo " + s + p
    #print(cmd)
    proc = subprocess.Popen([cmd],stdout=subprocess.PIPE,shell=True)
    (out, err) = proc.communicate()
    return out 

In [4]:
TauValues=pandas.DataFrame(data={'mRNAseq':[]})
Seq=pandas.DataFrame(data={'mRNAseq':["AACCAAACACACAAACGCACGGAAGGGUCAAAUAAUCCUUCUGGAAUCUUAACAGAGGAGAAAGAUUAUGGAAGGAUUA"]})


for index in range(0, 1000):
    NewVal=Seq.applymap(kinfold)
    TauValues=TauValues.append(NewVal)


TrajectoryTimes=pandas.DataFrame(data={'Time':[]})
for index in range (0, 1000):
    string=TauValues.iloc[index]['mRNAseq']
    if not string:
        val=10000000
    else:
        val=float(string.decode())
    NewVal=pandas.DataFrame(data={'Time':[val]})
    TrajectoryTimes=TrajectoryTimes.append(NewVal)

echo "AACCAAACACACAAACGCACGGAAGGGUCAAAUAAUCCUUCUGGAAUCUUAACAGAGGAGAAAGAUUAUGGAAGGAUUA"| /usr/local/bin/Kinfold --time 10000000 | grep X1 | perl -ne 'chomp;@a=split(/\s+/);print $a[2] . "\n";' | perl -e '@a=<>;$s=0;for(my $i=0;$i<scalar(@a);$i++){$s+=$a[$i];}print $s/scalar(@a);'
echo "AACCAAACACACAAACGCACGGAAGGGUCAAAUAAUCCUUCUGGAAUCUUAACAGAGGAGAAAGAUUAUGGAAGGAUUA"| /usr/local/bin/Kinfold --time 10000000 | grep X1 | perl -ne 'chomp;@a=split(/\s+/);print $a[2] . "\n";' | perl -e '@a=<>;$s=0;for(my $i=0;$i<scalar(@a);$i++){$s+=$a[$i];}print $s/scalar(@a);'
echo "AACCAAACACACAAACGCACGGAAGGGUCAAAUAAUCCUUCUGGAAUCUUAACAGAGGAGAAAGAUUAUGGAAGGAUUA"| /usr/local/bin/Kinfold --time 10000000 | grep X1 | perl -ne 'chomp;@a=split(/\s+/);print $a[2] . "\n";' | perl -e '@a=<>;$s=0;for(my $i=0;$i<scalar(@a);$i++){$s+=$a[$i];}print $s/scalar(@a);'
echo "AACCAAACACACAAACGCACGGAAGGGUCAAAUAAUCCUUCUGGAAUCUUAACAGAGGAGAAAGAUUAUGGAAGGAUUA"| /usr/local/bin/Kinfold --time 10000000 | grep X1 | perl -ne 'chomp;@a=sp

In [5]:
print(TrajectoryTimes)

          Time
0     9742.395
0      163.667
0     8367.520
0  6741318.580
0   603426.663
